Consider the following interval scheduling problem. We have $n$ intervals in time, each with a start time in $s\in [T]$ and an end time $e\in[T]$ with $s\leq e$. Each interval $I$ has a cost $c_I$ and a size $w_I$. There is a demand $d_t$ at each $t\in[T]$ and we assume $\sum_{I:t\in I}w_I \geq d_t$. The goal is to choose a subset $\mathcal I$ of the intervals to minimize $\sum_{I\in\mathcal I}c_I$ such that at each time $t$, $\sum_{I\in\mathcal I:t\in I}w_I\geq d_t$.

We can formulate this with the following linear program relaxation with knapsack-cover constraints. At each time $t$ and for each $A\subseteq\{I:t\in I\}$, denote $d_t(A) = d_t - w(A)$ where $w(A) = \sum_{I\in A}w_I$. This is the residual demand at $t$, assuming we take all intervals in $A$. Let $\tilde w_I^{t,A} = \min\{w_I, d_t(A)$. Now the linear program is to minimize $\sum_I c_Ix_I$ subject to $\sum_{I\notin A:t\in I}\tilde w_I^{t,A}x_I\geq d_t(A)$ for every $t$ and $A$, and $x_I\geq 0$ for each $I$.

I'm interested in the integrality gap of this linear programming relaxation, where for the integer solution, I want to enforce $x_I\in\{0,1\}$. Can you help me find this by writing me python code, to randomly generate instances, and find the ratio between the values of the integer and fractional solutions?

In [8]:
import random
import itertools
import pulp
from tqdm import tqdm

# -----------------------------
# 1. Instance Generation
# -----------------------------

def generate_instance(T=10, n=15, max_demand=10, max_cost=20, max_weight=10):
    intervals = []
    for _ in range(n):
        s = random.randint(1, T)
        e = random.randint(s, T)
        c = random.randint(1, max_cost)
        w = random.randint(1, max_weight)
        intervals.append({'s': s, 'e': e, 'c': c, 'w': w})
    d_t = {t: random.randint(1, max_demand) for t in range(1, T+1)}
    return intervals, d_t

def relevant_intervals(intervals, t):
    return [i for i, I in enumerate(intervals) if I['s'] <= t <= I['e']]

# -----------------------------
# 2. LP and IP Solver
# -----------------------------

def solve_lp(intervals, d_t, integrality=False):
    prob = pulp.LpProblem("IntervalScheduling", pulp.LpMinimize)
    x = [pulp.LpVariable(f'x_{i}', lowBound=0, upBound=1, cat='Binary' if integrality else 'Continuous')
         for i in range(len(intervals))]

    # Objective function
    prob += pulp.lpSum(intervals[i]['c'] * x[i] for i in range(len(intervals)))

    # Knapsack-cover constraints
    for t in d_t:
        J_t = relevant_intervals(intervals, t)
        for r in range(len(J_t)+1):
            for A in itertools.combinations(J_t, r):
                w_A = sum(intervals[i]['w'] for i in A)
                d_res = d_t[t] - w_A
                if d_res <= 0:
                    continue
                coeffs = []
                for i in set(J_t) - set(A):
                    w_i = intervals[i]['w']
                    coeffs.append((min(w_i, d_res), x[i]))
                if coeffs:
                    prob += (pulp.lpSum(c * var for c, var in coeffs) >= d_res)

    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    return pulp.value(prob.objective)

# -----------------------------
# 3. Run Multiple Experiments
# -----------------------------

def run_experiments(num_trials=20, T=10, n=12, seed=0):
    random.seed(seed)
    gaps = []
    best_gap = -1
    best_instance = None

    for _ in tqdm(range(num_trials), desc="Running experiments"):
        intervals, d_t = generate_instance(T=T, n=n)
        lp_val = solve_lp(intervals, d_t, integrality=False)
        ip_val = solve_lp(intervals, d_t, integrality=True)

        if lp_val is None or lp_val == 0:
            continue  # Skip degenerate cases

        gap = ip_val / lp_val
        gaps.append(gap)

        if gap > best_gap:
            best_gap = gap
            best_instance = {
                'intervals': intervals,
                'd_t': d_t,
                'lp_val': lp_val,
                'ip_val': ip_val,
                'gap': gap
            }

    if gaps:
        print(f"\n✅ Ran {len(gaps)} valid experiments.")
        print(f"📈 Maximum integrality gap: {best_instance['gap']:.3f}")
        print(f"📊 Average integrality gap: {sum(gaps)/len(gaps):.3f}")

        # Print the instance with max gap
        print("\n--- 🧪 Worst-case Instance ---")
        print(f"LP value: {best_instance['lp_val']:.3f}")
        print(f"IP value: {best_instance['ip_val']:.3f}")
        print(f"Gap: {best_instance['gap']:.3f}")

        print("\nIntervals:")
        for i, I in enumerate(best_instance['intervals']):
            print(f"  I{i}: start={I['s']}, end={I['e']}, cost={I['c']}, size={I['w']}")

        print("\nDemand profile:")
        for t in sorted(best_instance['d_t']):
            print(f"  t={t}: demand={best_instance['d_t'][t]}")
    else:
        print("⚠️ No valid LP values found.")

    return gaps, best_instance

# -----------------------------
# 4. Run the Script
# -----------------------------

if __name__ == "__main__":
    gaps, worst_instance = run_experiments(num_trials=20000, T=10, n=12)


Running experiments: 100%|████████████████| 20000/20000 [14:17<00:00, 23.32it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 3.238
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 21.000
IP value: 68.000
Gap: 3.238

Intervals:
  I0: start=8, end=9, cost=9, size=3
  I1: start=9, end=10, cost=7, size=4
  I2: start=10, end=10, cost=3, size=8
  I3: start=9, end=9, cost=12, size=10
  I4: start=6, end=10, cost=11, size=10
  I5: start=5, end=6, cost=18, size=10
  I6: start=7, end=8, cost=17, size=5
  I7: start=2, end=4, cost=3, size=5
  I8: start=8, end=8, cost=17, size=4
  I9: start=3, end=8, cost=11, size=2
  I10: start=4, end=8, cost=5, size=1
  I11: start=1, end=6, cost=13, size=3

Demand profile:
  t=1: demand=9
  t=2: demand=4
  t=3: demand=6
  t=4: demand=9
  t=5: demand=10
  t=6: demand=7
  t=7: demand=2
  t=8: demand=6
  t=9: demand=10
  t=10: demand=8


In [9]:
run_experiments(num_trials=20000, T=8, n=13)

Running experiments: 100%|████████████████| 20000/20000 [14:26<00:00, 23.08it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 2.768
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 18.667
IP value: 51.667
Gap: 2.768

Intervals:
  I0: start=4, end=4, cost=2, size=6
  I1: start=1, end=1, cost=20, size=1
  I2: start=4, end=5, cost=3, size=6
  I3: start=2, end=2, cost=10, size=6
  I4: start=4, end=8, cost=8, size=4
  I5: start=8, end=8, cost=6, size=1
  I6: start=5, end=5, cost=17, size=9
  I7: start=6, end=6, cost=8, size=5
  I8: start=1, end=2, cost=15, size=1
  I9: start=6, end=8, cost=18, size=9
  I10: start=4, end=7, cost=19, size=1
  I11: start=3, end=4, cost=1, size=5
  I12: start=8, end=8, cost=6, size=5

Demand profile:
  t=1: demand=2
  t=2: demand=5
  t=3: demand=9
  t=4: demand=8
  t=5: demand=2
  t=6: demand=4
  t=7: demand=4
  t=8: demand=4


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0497237569060773,
  1.0,
  1.0,
  1.0,
  1.0434782604158792,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.2325581402325583,
  1.0,
  1.1183431969206261,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0666666666666667,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.173913043478261,
  1.0,
  1.0714285714285714,
  1.0,
  1.0638297872340425,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1176470588235294,
  1.0,
  1.0,
  0.9999999999999996,
  1.0,
  1.1014492753623188,
  1.0,
  1.0,
  1.0,
  1.0909090909090908,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0644067771307095,
  1.0,
  1.0625,
  1.0,
  1.0,
  1.0,
  1.0

In [11]:
run_experiments(num_trials=20000, T=14, n=9)

Running experiments: 100%|████████████████| 20000/20000 [14:00<00:00, 23.80it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 2.713
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 13.500
IP value: 36.625
Gap: 2.713

Intervals:
  I0: start=13, end=14, cost=10, size=8
  I1: start=12, end=13, cost=13, size=2
  I2: start=6, end=13, cost=7, size=8
  I3: start=13, end=13, cost=5, size=6
  I4: start=3, end=7, cost=11, size=2
  I5: start=10, end=14, cost=10, size=2
  I6: start=12, end=14, cost=9, size=6
  I7: start=12, end=14, cost=13, size=10
  I8: start=4, end=9, cost=1, size=1

Demand profile:
  t=1: demand=9
  t=2: demand=1
  t=3: demand=3
  t=4: demand=7
  t=5: demand=6
  t=6: demand=9
  t=7: demand=10
  t=8: demand=1
  t=9: demand=10
  t=10: demand=1
  t=11: demand=3
  t=12: demand=7
  t=13: demand=1
  t=14: demand=3


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0991735537190082,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0642570309164368,
  1.0663507114058535,
  1.0,
  1.0849256904025855,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1315068489430664,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.018181818181818,
  1.0,
  1.0,
 

In [12]:
run_experiments(num_trials=20000, T=10, n=13)

Running experiments: 100%|████████████████| 20000/20000 [14:27<00:00, 23.06it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 3.500
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 2.000
IP value: 7.000
Gap: 3.500

Intervals:
  I0: start=7, end=10, cost=5, size=9
  I1: start=9, end=10, cost=1, size=7
  I2: start=2, end=9, cost=1, size=10
  I3: start=3, end=7, cost=16, size=9
  I4: start=9, end=9, cost=2, size=4
  I5: start=9, end=9, cost=5, size=5
  I6: start=4, end=9, cost=4, size=4
  I7: start=1, end=7, cost=2, size=2
  I8: start=8, end=9, cost=1, size=3
  I9: start=5, end=7, cost=18, size=2
  I10: start=8, end=9, cost=10, size=2
  I11: start=3, end=8, cost=6, size=4
  I12: start=9, end=9, cost=4, size=10

Demand profile:
  t=1: demand=5
  t=2: demand=4
  t=3: demand=9
  t=4: demand=4
  t=5: demand=1
  t=6: demand=10
  t=7: demand=4
  t=8: demand=10
  t=9: demand=4
  t=10: demand=5


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1525423749497272,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.296943231771324,
  1.0,
  1.0,
  1.0,
  1.0714285714285714,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0526315789473684,
  1.0,
  1.012307701740434,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0416666666666667,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0388349514563107,
  1.0,
  1.0,
  1.0896226420748043,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0127388533741735,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0267857142857142,
  1.1912350597609562,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.

In [13]:
run_experiments(num_trials=20000, T=11, n=14)

Running experiments: 100%|████████████████| 20000/20000 [14:38<00:00, 22.77it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 2.091
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 11.000
IP value: 23.000
Gap: 2.091

Intervals:
  I0: start=3, end=5, cost=19, size=9
  I1: start=3, end=7, cost=7, size=4
  I2: start=7, end=7, cost=7, size=3
  I3: start=5, end=10, cost=18, size=9
  I4: start=3, end=8, cost=10, size=1
  I5: start=5, end=7, cost=11, size=1
  I6: start=10, end=11, cost=4, size=9
  I7: start=7, end=9, cost=7, size=5
  I8: start=11, end=11, cost=5, size=3
  I9: start=7, end=10, cost=8, size=9
  I10: start=3, end=4, cost=18, size=7
  I11: start=11, end=11, cost=1, size=9
  I12: start=1, end=9, cost=4, size=5
  I13: start=4, end=11, cost=20, size=7

Demand profile:
  t=1: demand=10
  t=2: demand=6
  t=3: demand=6
  t=4: demand=8
  t=5: demand=8
  t=6: demand=1
  t=7: demand=9
  t=8: demand=8
  t=9: demand=7
  t=10: demand=3
  t=11: demand=5


([1.0,
  1.0769230769230769,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.2158808990538699,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.103448275862069,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.016260162771278,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0669056170624736,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0476190485714285,
  1.0,
  1.0,
  1.0,
  1.0219780217534113,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0601542410266915,
  1.0,
  1.0,
  1.0588235294117647,
  1.0,
  1.0,
  1.0,
  1.0,

In [14]:
run_experiments(num_trials=20000, T=9, n=13)

Running experiments: 100%|████████████████| 20000/20000 [14:29<00:00, 23.01it/s]


✅ Ran 20000 valid experiments.
📈 Maximum integrality gap: 2.818
📊 Average integrality gap: 1.009

--- 🧪 Worst-case Instance ---
LP value: 37.000
IP value: 104.250
Gap: 2.818

Intervals:
  I0: start=8, end=9, cost=3, size=2
  I1: start=3, end=4, cost=4, size=4
  I2: start=7, end=8, cost=18, size=2
  I3: start=9, end=9, cost=15, size=8
  I4: start=3, end=6, cost=7, size=10
  I5: start=7, end=8, cost=9, size=2
  I6: start=3, end=3, cost=11, size=5
  I7: start=6, end=6, cost=10, size=5
  I8: start=1, end=8, cost=5, size=1
  I9: start=1, end=2, cost=3, size=4
  I10: start=8, end=9, cost=4, size=2
  I11: start=8, end=9, cost=1, size=1
  I12: start=9, end=9, cost=13, size=7

Demand profile:
  t=1: demand=2
  t=2: demand=9
  t=3: demand=9
  t=4: demand=5
  t=5: demand=8
  t=6: demand=8
  t=7: demand=10
  t=8: demand=9
  t=9: demand=3


([1.0,
  1.0769230769230769,
  1.0,
  1.0,
  1.1089918227423916,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.018181818181818,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0074626864919805,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0652173855923128,
  1.0,
  1.0,
  1.0,
  1.0666666666666667,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0537634425598337,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.6816143506927548,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0277777776686507,
  1.1022688815128445,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1776900279030063,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.1136363636363635,
  1.1475409836065573,
  1.0,
  1.0,
  1.0,
  1.0714285706632654,
  1.0,
